In [1]:
import pynq
import numpy as np
from tqdm import tqdm
import time
#from sklearn.metrics import average_precision_score, roc_auc_score

# 通过PYNQ将合成电路写入FPGA中
ol_loda = pynq.Overlay('./SCD_Inter_Full_16161616.bit')
# ol_loda = pynq.Overlay('./SCD_Inter_Half_16161616.bit')
mSCD = ol_loda.model_SCD_0
mM2M = ol_loda.Multi2Multi_0
mM2O = ol_loda.Multi2One_0

CONTROL_REGISTER = 0x0
mM2O.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED=0)
}

In [2]:
def _load_via_txt(path):
    X = np.loadtxt(path, delimiter=",")
    return X

def get_data(data_file):
    #data_path = os.path.join(data_base_path, data_file)
    data_path = data_file
    if ".mat" in data_file:
        from scipy.io import loadmat
        f = loadmat(data_path)
        X = f['X']
    else:
        X = _load_via_txt(data_path)
    return X

X_train = get_data("./deepsig.dat")  # Load Aryhytmia data.
print("X_train.shape = ",X_train.shape)

X_train.shape =  (2048, 2)


In [3]:
# 使用AXI DMA进行存储器传输
dma_i = ol_loda.axi_dma_0.sendchannel
dma_q = ol_loda.axi_dma_1.sendchannel
dma_v = ol_loda.axi_dma_2.recvchannel
dma_a = ol_loda.axi_dma_3.recvchannel
dma_f = ol_loda.axi_dma_4.recvchannel
# 处理大量数据
N = 2048 # X_train.shape[0]
Nout = 807#32768
NSamples = 1000

buff_x = pynq.allocate(shape=(NSamples*N,), dtype=np.float32)
buff_y = pynq.allocate(shape=(NSamples*N,), dtype=np.float32)
buff_value = pynq.allocate(shape=(NSamples*Nout,), dtype=np.float32)
buff_al = pynq.allocate(shape=(NSamples*Nout,), dtype=np.int16)
buff_fl = pynq.allocate(shape=(NSamples*Nout,), dtype=np.int16)

# print("X_train[0:N] = ",X_train[0:N][:])


for j in range (NSamples):
    buff_x[j*N:(j+1)*N] = X_train[0:N,0]
    buff_y[j*N:(j+1)*N] = X_train[0:N,1]
    

In [4]:
start = time.time()
mSCD.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
mM2M.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
mM2O.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0

dma_i.transfer(buff_x)
dma_q.transfer(buff_y)
dma_v.transfer(buff_value)
dma_a.transfer(buff_al)
dma_f.transfer(buff_fl)
dma_i.wait()
dma_q.wait()
# dma_r.wait()
end = time.time()
mSCD.write(CONTROL_REGISTER,0x00) # stop
mM2M.write(CONTROL_REGISTER,0x00) # stop
mM2O.write(CONTROL_REGISTER,0x00) # stop
time_cost= end-start
print('=> Running on FPGA')
print(time_cost)
print()

=> Running on FPGA
0.266477108001709



In [5]:
# print("buff_y[0:N] = ",buff_y[0:10])
# print("buff_y[-1] = ",buff_y[-1])
# del buff_x, buff_y

In [6]:
buff_value[0]

0.106536865

In [7]:
x = np.arange(0,809)
for i in x:
    print(i,":",buff_value[i],",",buff_al[i],",",buff_fl[i])
#     print(i,";",buff_z[i],",")

0 : 0.106536865 , -255 , 8
1 : 0.050323486 , -248 , 66
2 : 0.05001831 , -232 , 192
3 : 0.061035156 , -50 , 1648
4 : 0.099243164 , -253 , 8
5 : 0.0552063 , -48 , 1648
6 : 0.052734375 , -26 , 1824
7 : 0.06472778 , -247 , 8
8 : 0.050689697 , -244 , 32
9 : 0.07803345 , -243 , 8
10 : 0.07223511 , -22 , 1776
11 : 0.055603027 , -4 , 1920
12 : 0.0519104 , -244 , -16
13 : 0.23672485 , -241 , 8
14 : 0.05722046 , -235 , 56
15 : 0.06851196 , -195 , 376
16 : 0.051696777 , -158 , 672
17 : 0.053588867 , -153 , 712
18 : 0.06008911 , -150 , 736
19 : 0.056243896 , -139 , 824
20 : 0.053344727 , -131 , 888
21 : 0.060791016 , -125 , 938
22 : 0.059326172 , -117 , 1000
23 : 0.095825195 , -104 , 1104
24 : 0.051757812 , -69 , 1386
25 : 0.08465576 , -29 , 1704
26 : 0.053894043 , -21 , 1768
27 : 0.09802246 , -20 , 1776
28 : 0.0541687 , -9 , 1864
29 : 0.103271484 , -5 , 1896
30 : 0.052612305 , 2 , 1952
31 : 0.050323486 , -248 , -66
32 : 0.07015991 , -167 , 582
33 : 0.05117798 , -131 , 868
34 : 0.06048584 , -97 , 

347 : 0.060821533 , -118 , -768
348 : 0.050811768 , -96 , -592
349 : 0.053222656 , -92 , -560
350 : 0.060455322 , -57 , -278
351 : 0.15356445 , -21 , 8
352 : 0.056610107 , -8 , 112
353 : 0.0657959 , 6 , 224
354 : 0.06259155 , 52 , 592
355 : 0.061920166 , 67 , 712
356 : 0.07330322 , 71 , 744
357 : 0.056671143 , 89 , 888
358 : 0.050964355 , 90 , 896
359 : 0.06756592 , 107 , 1032
360 : 0.052703857 , 112 , 1072
361 : 0.06021118 , 116 , 1104
362 : 0.0602417 , -92 , -624
363 : 0.0718689 , -13 , 8
364 : 0.050811768 , -4 , 80
365 : 0.07876587 , -11 , 8
366 : 0.052490234 , -112 , -816
367 : 0.09249878 , -90 , -640
368 : 0.050689697 , -88 , -624
369 : 0.056549072 , -82 , -576
370 : 0.07675171 , -57 , -376
371 : 0.06112671 , -39 , -232
372 : 0.15737915 , -9 , 8
373 : 0.051574707 , 54 , 512
374 : 0.06201172 , 57 , 536
375 : 0.05899048 , 65 , 602
376 : 0.05407715 , 75 , 680
377 : 0.05731201 , 96 , 850
378 : 0.05255127 , 114 , 992
379 : 0.060791016 , -125 , -938
380 : 0.064971924 , -60 , -464
381 : 

633 : 0.22961426 , 139 , 8
634 : 0.059570312 , 188 , 402
635 : 0.05529785 , -43 , -1466
636 : 0.05380249 , 17 , -986
637 : 0.05899048 , 65 , -602
638 : 0.052703857 , 131 , -74
639 : 0.072509766 , 41 , -840
640 : 0.05105591 , 173 , 216
641 : 0.064208984 , -50 , -1632
642 : 0.06832886 , -43 , -1574
643 : 0.056427002 , -43 , -1576
644 : 0.080444336 , -4 , -1264
645 : 0.05038452 , 12 , -1134
646 : 0.072052 , 59 , -760
647 : 0.05596924 , 59 , -758
648 : 0.06552124 , 67 , -696
649 : 0.0513916 , 94 , -478
650 : 0.17541504 , 155 , 8
651 : 0.06109619 , 178 , 194
652 : 0.058288574 , 189 , 280
653 : 0.07196045 , 193 , 314
654 : 0.050231934 , 195 , 328
655 : 0.05316162 , -48 , -1632
656 : 0.050079346 , 11 , -1162
657 : 0.05480957 , 66 , -720
658 : 0.14144897 , 157 , 8
659 : 0.05154419 , -6 , -1312
660 : 0.053771973 , 75 , -664
661 : 0.07180786 , 159 , 8
662 : 0.055023193 , 179 , 166
663 : 0.05340576 , 161 , 8
664 : 0.07342529 , 71 , -728
665 : 0.08035278 , 163 , 8
666 : 0.06271362 , 182 , 160
667 

In [8]:
import re
import math
dat_file = r"sparse_SCD_float_full16161616.dat"   
dm = []
al = []
fl = []
with open(dat_file, 'r') as file:
    text = file.read()
    data = re.split(',|\n', text)
    for i in range(int((len(data)-1)/3)):
        dm.append(float(data[i*3]))
        al.append(int(data[i*3+1]))
        fl.append(int(data[i*3+2]))

In [9]:
rdm = np.zeros(len(dm))
ral = np.zeros(len(dm))
rfl = np.zeros(len(dm))

In [10]:
for i in range(len(dm)):
    for j in range(len(np.where(al==buff_al[i])[-1])):
        if fl[np.where(al==buff_al[i])[0][j]] == buff_fl[i]:
#             print(i)
#             print(buff_al[i])
#             print(np.where(al==buff_al[i])[0][j])
#             print(fl[np.where(al==buff_al[i])[0][j]])
#             print(buff_fl[i])
#             print(buff_value[i])
#             print(dm[np.where(al==buff_al[i])[0][j]])
#             print('------------')
            rdm[i]=dm[np.where(al==buff_al[i])[0][j]]
            ral[i]=al[np.where(al==buff_al[i])[0][j]]
            rfl[i]=fl[np.where(al==buff_al[i])[0][j]]

In [11]:
Psignal = np.sum(np.power(rdm,2))
# Pnoise  = np.var(rdm-buff_value[0:len(rdm)])
Pnoise  = np.sum(np.power(rdm-buff_value[0:len(rdm)],2))
print("The SQNR is: " , 10*math.log10(Psignal/Pnoise))

The SQNR is:  73.42506513933867


In [12]:
Psignal = np.sum(np.power(rdm,2))
Pnoise  = np.var(rdm-buff_value[0:len(rdm)])
print("The SQNR is: " , 10*math.log10(Psignal/Pnoise))

The SQNR is:  102.52857723063273


In [13]:
for i in range(len(dm)):
    print(str(i)+':'+str(rdm[i])+','+str(buff_value[i]))

0:0.106570199,0.106536865
1:0.050304554,0.050323486
2:0.050025627,0.05001831
3:0.061018914,0.061035156
4:0.099229634,0.099243164
5:0.055203404,0.0552063
6:0.052717499,0.052734375
7:0.064704522,0.06472778
8:0.050693706,0.050689697
9:0.078040242,0.07803345
10:0.072224282,0.07223511
11:0.055587776,0.055603027
12:0.051923238,0.0519104
13:0.2367336,0.23672485
14:0.057224158,0.05722046
15:0.068532616,0.06851196
16:0.051677741,0.051696777
17:0.053572781,0.053588867
18:0.060099266,0.06008911
19:0.056231719,0.056243896
20:0.053362302,0.053344727
21:0.060780633,0.060791016
22:0.059308823,0.059326172
23:0.095874965,0.095825195
24:0.051770981,0.051757812
25:0.084678628,0.08465576
26:0.053900957,0.053894043
27:0.098013066,0.09802246
28:0.0541641,0.0541687
29:0.103276707,0.103271484
30:0.052605934,0.052612305
31:0.050304554,0.050323486
32:0.070137717,0.07015991
33:0.051177617,0.05117798
34:0.060501561,0.06048584
35:0.051826145,0.051818848
36:0.068356007,0.06832886
37:0.054369535,0.054382324
38:0.064

In [14]:
np.max(rdm-buff_value[0:len(rdm)])

PynqBuffer(4.97696875e-05)

In [15]:
math.log2(np.max(rdm-buff_value[0:len(rdm)]))

-14.294373145963299

In [16]:
np.power([1,2,3],2)

array([1, 4, 9])

In [17]:
ral-buff_al[0:len(ral)]

PynqBuffer([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [18]:
rfl-buff_fl[0:len(rfl)]

PynqBuffer([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 